In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options


In [2]:
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd
import numpy as np
import re

In [3]:
from places import places_list
all_places = places_list()
base_link = "https://www.bikedekho.com/used-cars+"

FileNotFoundError: [Errno 2] No such file or directory: 'Scraper/Bike_Scrape/Bike_dekho/BikeDekho_places.html'

In [4]:
engine_types = [
    'upto-100cc',
    '100cc-125cc',
    '125cc-150cc',
    '150cc-200cc',
    '200cc-250cc',
    '250cc-500cc',
    '500cc-1000cc',
    'above-1000cc'
    ]


In [5]:
options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(options=options)

In [6]:
driver.get('https://www.bikedekho.com/used-cars+in+anantapur')

In [7]:
def load_till_last():
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        sleep(2)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    driver.execute_script("window.scrollTo(0, 100);")
    return None

In [8]:
def split_title_line(text):
    list1 = text.split(' ')

    try:
        if (re.search(r'[1-2][0-9][0-9][0-9]', text).span()[0] == 0):
            year = list1[0]
            list1.pop(0)
        else:
            year=np.nan
    except:
        year=np.nan

    try:    
        if (re.search(r'[a-zA-Z]', text).span()[0] == 5):
            company = list1[0]
            list1.pop(0)
        elif (re.search(r'[a-zA-Z]', text).span()[0] == 0):
            company = list1[0]
            list1.pop(0)
        else:
            company = 'others'
    except:
        company = 'others'

    if list1[0].lower() in ['enfield', 'davidson', 'honda',]:
        company+=' '+list1[0]
        list1.pop(0)

    model = ' '.join(list1)

    return year, company, model

In [9]:
def split_dot_list(span_list):
    i = 0
    if span_list[i].text.lstrip().rstrip().lower() not in ['petrol', 'electric', 'first', 'second']:
        kms_driven = span_list[i].text.lstrip().rstrip()
        i+=1
    else:
        kms_driven = np.nan
    if span_list[i].text.lstrip().rstrip()[3:].lower() in ['petrol', 'electric']:
        fuel_type = span_list[i].text.lstrip().rstrip()[3:]
        i+=1
        try:
            ownership = span_list[i].text.lstrip().rstrip()[3:]
        except:
            ownership = np.nan
    elif span_list[i].text.lstrip().rstrip()[3:].lower() in ['first', 'second']:
        fuel_type = np.nan
        ownership = span_list[i].text.lstrip().rstrip()[3:]
    
    # ownership = span_list[i].text.lstrip().rstrip()[3:]
    return kms_driven, fuel_type, ownership

In [10]:
def get_price(price_element):
    price_element.find('sup').decompose()
    price_element.find('span', {'class':'icon-cd_R'}).decompose()
    return price_element.text.rstrip().lstrip()

In [11]:
def get_details(bike_element):
    html_cntnt = bike_element.get_attribute('outerHTML') 
    soup = BeautifulSoup(html_cntnt, 'html.parser')
    kms_driven, fuel_type, ownership = split_dot_list(
        soup.find(
            'div', 
            {'class':'dotlist'}
            ).findChildren()
        )
    price = get_price(
        soup.find(
            'div', 
            {'class':'price'})
        )
    year, company, model = split_title_line(
        soup.find(
            'div', 
            {'class':'title'}
            ).text.rstrip().lstrip()
        )

    return company, model,year,kms_driven,fuel_type,ownership,price



In [16]:
Xpath_to_engine_type = '//*[@id="rf01"]/div[1]/div/main/div[1]/div[2]/div/div[4]/div[6]/div/ul/'
# //*[@id="rf01"]/div[1]/div/main/div[1]/div[2]/div/div[4]/div[5]/div/ul/li[4]/label
# cc_types = [i.text for i in driver.find_elements(By.XPATH, f'{Xpath_to_engine_type}li')]
# driver.find_element(By.XPATH, f'{Xpath_to_engine_type}li[4]/label').click()
sleep(5)
driver.find_element(By.XPATH, f'{Xpath_to_engine_type}li[4]/label').click()
if 'cc' in driver.find_element(By.XPATH, f'{Xpath_to_engine_type}/li').text:
    print('true')

true


In [13]:
def get_through_cc(cc, cc_num, place, dictionary):
    driver.find_element(By.XPATH, f'{Xpath_to_engine_type}li[{str(cc_num)}]/label').click()
    sleep(3)
    load_till_last()
    
    list_of_bikes_element = driver.find_elements(By.CLASS_NAME, 'reportAd')
    
    print(len(list_of_bikes_element))
    print(cc)
    for bike_element in list_of_bikes_element:
        company, model,year,kms_driven,fuel_type,ownership,price = get_details(bike_element)
        dictionary['company'].append(company)
        dictionary['model'].append(model)
        dictionary['year'].append(year)
        dictionary['kms_driven'].append(kms_driven)
        dictionary['fuel_type'].append(fuel_type)
        dictionary['ownership'].append(ownership)
        dictionary['price'].append(price)
        dictionary['cc_type'].append(cc)
        dictionary['place'].append(place)
        
    driver.find_element(By.XPATH, f'{Xpath_to_engine_type}li[{str(cc_num)}]/label').click()
    return dictionary



In [14]:
[i.text for i in driver.find_elements(By.XPATH, f'{Xpath_to_engine_type}li')]

['Upto 100 cc(13)',
 '100 - 125 cc(22)',
 '125 - 150 cc(15)',
 '150 - 200 cc(21)',
 '200 - 250 cc(3)',
 '250 - 500 cc(12)']

In [15]:
def create_dict():
    return {
            'company':[], 
            'model': [],
            'year':[], 
            'kms_driven':[], 
            'cc_type':[],
            'fuel_type':[], 
            'place':[],
            'ownership':[], 
            'price':[],
        }

In [16]:
def get_through_area(place):
    driver.execute_script("window.open('');")
    driver.switch_to.window(driver.window_handles[1])
    link = base_link+'in+'+place
    driver.get(link)
    sleep(3)
    
    df = pd.DataFrame(columns=['company', 'model', 'year', 'kms_driven', 'cc_type', 'fuel_type', 'place', 'ownership', 'price'], index=None)
    
    cc_types = [i.text for i in driver.find_elements(By.XPATH, f'{Xpath_to_engine_type}li')]
    num=1
    for cc in cc_types:
        dictionary = create_dict()
        dictionary = get_through_cc(
            cc, 
            num,
            place, 
            dictionary
            )
        num+=1
        df = pd.concat([df, pd.DataFrame(dictionary, index=None)])
        del dictionary
        sleep(2)
    
    # Switching to old tab
    driver.close()
    driver.switch_to.window(driver.window_handles[0])

    return df

In [17]:
# len(all_places)
df = pd.DataFrame()
for place in all_places[40:45]:
    df = pd.concat([df,get_through_area(place.replace(' ', '-').lower())])
# get_through_area('ahmedabad')

1
125 - 150 cc(1)
1
150 - 200 cc(1)
1
250 - 500 cc(1)
1
100 - 125 cc(1)
1
125 - 150 cc(1)
1
125 - 150 cc(1)
1
150 - 200 cc(1)
2
250 - 500 cc(2)
1
Upto 100 cc(1)
1
150 - 200 cc(1)
1
Upto 100 cc(1)
1
125 - 150 cc(1)


In [18]:
df.to_csv('BikeDekho.csv', index=None)

In [ ]:
df.to_csv()